# Train Toxicity Model

This notebook trains a model to detect toxicity in online comments. It uses a CNN architecture for text classification trained on the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973) and pre-trained GloVe embeddings which can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/).

This model is a modification of [example code](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) found in the [Keras Github repository](https://github.com/fchollet/keras) and released under an [MIT license](https://github.com/fchollet/keras/blob/master/LICENSE). For further details of this license, find it [online](https://github.com/fchollet/keras/blob/master/LICENSE) or in this repository in the file KERAS_LICENSE. 

## Usage Instructions
(TODO: nthain) - Move to README

Prior to running the notebook, you must:

* Download the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973)
* Download pre-trained [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip)
* (optional) To skip the training step, you will need to download a model and tokenizer file. We are looking into the appropriate means for distributing these (sometimes large) files.

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd

from model_tool import ToxModel
from attention_model import AttentionToxModel

Using TensorFlow backend.


HELLO from model_tool


## Load Data

In [3]:
SPLITS = ['train', 'dev', 'test']

wiki = {}
debias = {}
random = {}
for split in SPLITS:
    wiki[split] = '../data/wiki_%s.csv' % split
    debias[split] = '../data/wiki_debias_%s.csv' % split
    random[split] = '../data/wiki_debias_random_%s.csv' % split
    
print(dir(AttentionToxModel))

['__doc__', '__init__', '__module__', 'build_conv_layer', 'build_dense_attention_layer', 'build_model', 'fit_and_save_tokenizer', 'get_model_name', 'load_embeddings', 'load_model_from_name', 'predict', 'prep_text', 'print_hparams', 'save_hparams', 'score_auc', 'summary', 'train', 'update_hparams']


## Train Models

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import cPickle
import json
import os
import numpy as np
import pandas as pd

from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn import metrics

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam


print('HELLO from model_tool')

DEFAULT_EMBEDDINGS_PATH = '../data/glove.6B/glove.6B.100d.txt'
DEFAULT_MODEL_DIR = '../models'

DEFAULT_HPARAMS = {
    'max_sequence_length': 250,
    'max_num_words': 10000,
    'embedding_dim': 100,
    'embedding_trainable': False,
    'learning_rate': 0.00005,
    'stop_early': True,
    'es_patience': 1, # Only relevant if STOP_EARLY = True
    'es_min_delta': 0, # Only relevant if STOP_EARLY = True
    'batch_size': 128,
    'epochs': 20,
    'dropout_rate': 0.3,
    'cnn_filter_sizes': [128, 128, 128],
    'cnn_kernel_sizes': [5,5,5],
    'cnn_pooling_sizes': [5, 5, 40],
    'verbose': True
}


def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan


class ToxModel:
    def __init__(self, 
                 model_name = None, 
                 model_dir = DEFAULT_MODEL_DIR,
                 hparams = None):
        self.model_dir = model_dir
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.hparams = DEFAULT_HPARAMS.copy()
        if hparams:
            self.update_hparams(hparams)
        if model_name:
            self.load_model_from_name(model_name)
        self.print_hparams()

    def print_hparams(self):
        print('Hyperparameters')
        print('---------------')
        for k, v in self.hparams.iteritems():
            print('{}: {}'.format(k, v))
        print('')

    def update_hparams(self, new_hparams):
        self.hparams.update(new_hparams)

    def get_model_name(self):
        return self.model_name

    def save_hparams(self, model_name):
        self.hparams['model_name'] = model_name
        with open(os.path.join(self.model_dir, 
                '%s_hparams.json' % self.model_name), 'w') as f:
            json.dump(self.hparams, f, sort_keys=True)

    def load_model_from_name(self, model_name):
        self.model = load_model(os.path.join(self.model_dir, '%s_model.h5' % model_name))
        self.tokenizer = cPickle.load(open(os.path.join(self.model_dir, 
                                                        '%s_tokenizer.pkl' % model_name), 
                                           'rb'))
        with open(os.path.join(self.model_dir, 
                '%s_hparams.json' % self.model_name), 'r') as f:
            self.hparams = json.load(f)

    def fit_and_save_tokenizer(self, texts):
        """Fits tokenizer on texts and pickles the tokenizer state."""
        self.tokenizer = Tokenizer(num_words = self.hparams['max_num_words'])
        self.tokenizer.fit_on_texts(texts)
        cPickle.dump(self.tokenizer, open(os.path.join(self.model_dir, '%s_tokenizer.pkl' % self.model_name), 'wb'))

    def prep_text(self, texts):
        """Turns text into into padded sequences.

        The tokenizer must be initialized before calling this method.

        Args:
            texts: Sequence of text strings.

        Returns:
            A tokenized and padded text sequence as a model input.
        """
        text_sequences = self.tokenizer.texts_to_sequences(texts)
        return pad_sequences(text_sequences, maxlen=self.hparams['max_sequence_length'])

    def load_embeddings(self, embedding_path = DEFAULT_EMBEDDINGS_PATH):
        embeddings_index = {}
        with open(embedding_path) as f:
            for line in f:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs

        self.embedding_matrix = np.zeros((len(self.tokenizer.word_index) + 1, self.hparams['embedding_dim']))
        num_words_in_embedding = 0
        for word, i in self.tokenizer.word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                num_words_in_embedding += 1
                # words not found in embedding index will be all-zeros.
                self.embedding_matrix[i] = embedding_vector

    def train(self, training_data_path, validation_data_path, text_column, label_column, model_name):
        self.model_name = model_name
        self.save_hparams(model_name)

        train_data = pd.read_csv(training_data_path)
        valid_data = pd.read_csv(validation_data_path)

        print('Fitting tokenizer...')
        self.fit_and_save_tokenizer(train_data[text_column])
        print('Tokenizer fitted!')

        print('Preparing data...')
        train_text, train_labels = (self.prep_text(train_data[text_column]),
                                    to_categorical(train_data[label_column]))
        valid_text, valid_labels = (self.prep_text(valid_data[text_column]),
                                    to_categorical(valid_data[label_column]))
        print('Data prepared!')

        print('Loading embeddings...')
        self.load_embeddings()
        print('Embeddings loaded!')

        print('Building model graph...')
        self.build_model()
        print('Training model...')

        save_path = os.path.join(self.model_dir, '%s_model.h5' % self.model_name)
        callbacks = [ModelCheckpoint(save_path, save_best_only=True, verbose=self.hparams['verbose'])]

        if self.hparams['stop_early']:
            callbacks.append(EarlyStopping(min_delta=self.hparams['es_min_delta'],
                monitor='val_loss', patience=self.hparams['es_patience'], verbose=self.hparams['verbose'], mode='auto'))

        
        self.model.fit(train_text,
                       train_labels,
                       batch_size=self.hparams['batch_size'],
                       epochs=self.hparams['epochs'],
                       validation_data=(valid_text, valid_labels),
                       callbacks=callbacks,
                       verbose=2)
        
        print('Model trained!')
        print('Best model saved to {}'.format(save_path))
        print('Loading best model from checkpoint...')
        self.model = load_model(save_path)
        print('Model loaded!')

        if self.probs_model:
            print('Fitting probs model')
            save_path = os.path.join(self.model_dir, 'probs_model.h5')
            callbacks = [ModelCheckpoint(save_path, save_best_only=True, verbose=self.hparams['verbose'])]

            self.probs_model.fit(train_text,
                       train_labels,
                       batch_size=self.hparams['batch_size'],
                       epochs=self.hparams['epochs'],
                       validation_data=(valid_text, valid_labels),
                       callbacks=callbacks,
                       verbose=2)
            
            self.probs_model = load_model(save_path)
            print('probs model loaded')
            
    def build_model(self):
        sequence_input = Input(shape=(self.hparams['max_sequence_length'],), dtype='int32')
        embedding_layer = Embedding(len(self.tokenizer.word_index) + 1,
                                    self.hparams['embedding_dim'],
                                    weights=[self.embedding_matrix],
                                    input_length=self.hparams['max_sequence_length'],
                                    trainable=self.hparams['embedding_trainable'])

        embedded_sequences = embedding_layer(sequence_input)
        x = embedded_sequences
        for filter_size, kernel_size, pool_size in zip(self.hparams['cnn_filter_sizes'], self.hparams['cnn_kernel_sizes'], self.hparams['cnn_pooling_sizes']):
            x = self.build_conv_layer(x, filter_size, kernel_size, pool_size)

        x = Flatten()(x)
        x = Dropout(self.hparams['dropout_rate'])(x)
        # TODO(nthain): Parametrize the number and size of fully connected layers
        x = Dense(128, activation='relu')(x)
        preds = Dense(2, activation='softmax')(x)

        rmsprop = RMSprop(lr = self.hparams['learning_rate'])
        self.model = Model(sequence_input, preds)
        self.model.compile(loss='categorical_crossentropy',
                      optimizer=rmsprop,
                      metrics=['acc'])
                

    def build_conv_layer(self, input_tensor, filter_size, kernel_size, pool_size):
        output = Conv1D(filter_size, kernel_size, activation='relu', padding='same')(input_tensor)
        if pool_size:
            output = MaxPooling1D(pool_size, padding = 'same')(output)
        else:
            # TODO(nthain): This seems broken. Fix.
            output = GlobalMaxPooling1D()(output)
        return output

    def predict(self, texts):
        """Returns model predictions on texts."""
        data = self.prep_text(texts)
        return self.model.predict(data)[:,1]

    def score_auc(self, texts, labels):
        preds = self.predict(texts)
        return compute_auc(labels, preds)


    def summary():
        return self.model.summary()



HELLO from model_tool


In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Dropout, merge, Multiply
from keras.models import Model
from keras.optimizers import RMSprop

attention_probs = None
attention_mul = None
attention_input = None

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

class AttentionToxModel(ToxModel):

    def build_dense_attention_layer(self, input_tensor):
        attention_input = input_tensor
        attention_probs = Dense(self.hparams['max_sequence_length'], activation='softmax', name='attention_vec')(input_tensor)
        attention_mul = Multiply()([input_tensor, attention_probs])
        return {'attention_probs':attention_probs, 'attention_preds':attention_mul}

    def build_probs(self):
        sequence_input = Input(shape=(self.hparams['max_sequence_length'],), dtype='int32')
        embedding_layer = Embedding(len(self.tokenizer.word_index) + 1,
                                    self.hparams['embedding_dim'],
                                    weights=[self.embedding_matrix],
                                    input_length=self.hparams['max_sequence_length'],
                                    trainable=self.hparams['embedding_trainable'])

        embedded_sequences = embedding_layer(sequence_input)
        x = embedded_sequences
        for filter_size, kernel_size, pool_size in zip(self.hparams['cnn_filter_sizes'], self.hparams['cnn_kernel_sizes'], self.hparams['cnn_pooling_sizes']):
            x = self.build_conv_layer(x, filter_size, kernel_size, pool_size)

        x = Flatten()(x)
        x = Dropout(self.hparams['dropout_rate'], name="Dropout")(x)
        # TODO(nthain): Parametrize the number and size of fully connected layers
        x = Dense(250, activation='relu', name="Dense_RELU")(x)

        attention_dict = self.build_dense_attention_layer(x)
        preds = attention_dict['attention_probs']
        preds = Dense(2, name="preds_dense")(preds)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.model = Model(sequence_input, preds)
        self.model.compile(loss='categorical_crossentropy', optimizer=rmsprop,metrics=['acc'])

    def build_model(self):
        print('print inside build model')
        sequence_input = Input(shape=(self.hparams['max_sequence_length'],), dtype='int32')
        embedding_layer = Embedding(len(self.tokenizer.word_index) + 1,
                                    self.hparams['embedding_dim'],
                                    weights=[self.embedding_matrix],
                                    input_length=self.hparams['max_sequence_length'],
                                    trainable=self.hparams['embedding_trainable'])

        embedded_sequences = embedding_layer(sequence_input)
        x = embedded_sequences
        for filter_size, kernel_size, pool_size in zip(self.hparams['cnn_filter_sizes'], self.hparams['cnn_kernel_sizes'], self.hparams['cnn_pooling_sizes']):
            x = self.build_conv_layer(x, filter_size, kernel_size, pool_size)

        x = Flatten()(x)
        x = Dropout(self.hparams['dropout_rate'], name="Dropout")(x)
        # TODO(nthain): Parametrize the number and size of fully connected layers
        x = Dense(250, activation='relu', name="Dense_RELU")(x)

        attention_dict = self.build_dense_attention_layer(x)
        preds = attention_dict['attention_preds']
        preds = Dense(2, name="preds_dense", activation='softmax')(preds)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.model = Model(sequence_input, preds)
        self.model.compile(loss='categorical_crossentropy', optimizer=rmsprop,metrics=['acc'])

        # now make probs model
        probs = attention_dict['attention_probs']
        probs = Dense(2, name='probs_dense')(probs)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.probs_model = Model(sequence_input, preds)
        self.probs_model.compile(loss='mse', optimizer=rmsprop,metrics=['acc'])

In [6]:
from keras_attention.data.reader import Vocabulary, INPUT_PADDING, OUTPUT_PADDING
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def attention_map(text, pred_model, proba_model, input_vocab, output_vocab):
    """
    Text to visualze attention map for.
    """
    # encode the string
    d = input_vocab.string_to_int(text)

    # get the output sequence
    predicted_text = run_example(pred_model, input_vocab, output_vocab, text)

    text_ = list(text) + ['<eot>'] + ['<unk>'] 
    # get the lengths of the string
    input_length = len(text)+1
    output_length = predicted_text.index('<eot>')+ input_vocab.padding
    # get the activation map
    activation_map = np.squeeze(proba_model.predict(np.array([d])))[
        0:output_length, 0:input_length]

    # import seaborn as sns
    plt.clf()
    f = plt.figure(figsize=(8, 8.5))
    ax = f.add_subplot(1, 1, 1)

    # add image
    i = ax.imshow(activation_map, interpolation='nearest', cmap='gray')
        
    # add colorbar
    cbaxes = f.add_axes([0.2, 0, 0.6, 0.03])
    cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
    cbar.ax.set_xlabel('Probability', labelpad=2)

    # add labels
    ax.set_yticks(range(output_length))
    ax.set_yticklabels(predicted_text[:output_length])
        
    ax.set_xticks(range(input_length))
    ax.set_xticklabels(text_[:input_length], rotation=45)
        
    ax.set_xlabel('Input Sequence')
    ax.set_ylabel('Output Sequence')

    # add grid and legend
    ax.grid()
    # ax.legend(loc='best')

    f.savefig(os.path.join(HERE, 'attention_maps', text.replace('/', '')+'.pdf'), bbox_inches='tight')
    f.show()

    


ImportError: No module named keras_attention.data.reader

## Attention Tox Model

In [16]:
model = AttentionToxModel(model_name='cnn_attention_random_tox_v5')


Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
embedding_dim: 100
cnn_kernel_sizes: [5, 5, 5]
es_patience: 1
epochs: 20
cnn_filter_sizes: [128, 128, 128]
batch_size: 128
model_name: cnn_attention_random_tox_v5
max_sequence_length: 250
stop_early: True
embedding_trainable: False



In [33]:
random_test = pd.read_csv(random['test'])
print(random_test)
model.score_auc(random_test['comment'], random_test['is_toxic'])

                                                 comment  is_toxic logged_in  \
0        == use of clown triggerfish ==  Dear Derek, ...     False     False   
1      ` :::Regardless of whatever the supposed ``mai...     False      True   
2      `  ==Wishaw General Hospital== A {{prod}} temp...     False      True   
3       (UTC) * Flavour (particle physics) → Flavor (...     False      True   
4                             ==SD.net VfD== Reverted.       False      True   
5      `  == wanted: location of the theorem in Hamil...     False      True   
6      `  ::::::::What you are missing is that we sim...     False      True   
7       :You probably don't know it but you helped me...     False      True   
8        The Washington Post is a reliable source, th...     False      True   
9       Also I am party to someone's interview with W...     False      True   
10     The last surviving Companion of the Order, Vic...     False       NaN   
11      :.  It should not be added unles

0.92785887000837941

In [22]:

MODEL_NAME = 'cnn_attention_random_tox_v5'
debias_attention_model = AttentionToxModel()
debias_attention_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.23587, saving model to ../models/cnn_attention_random_tox_v5_model.h5
9s - loss: 0.3008 - acc: 0.9061 - val_loss: 0.2359 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved from 0.23587 to 0.20001, saving model to ../models/cnn_attention_random_tox_v5_model.h5
9s - loss: 0.2154 - acc: 0.9108 - val_loss: 0.2000 - val_acc: 0.9300
Epoch 3/20
Epoch 00002: val_los

In [21]:
#debias_attention_model_probs_model = ToxModel('probs')
text = 'transit'
#text = 'i love you'
comment = debias_attention_model.prep_text(text)
a = debias_attention_model.predict(text)
print(a)
print(len(text))
#ata = self.prep_text(texts)
#eturn self.model.predict(data)[:,1]
# you will be able to graph percent toxic of each character
pm= debias_attention_model.probs_model.predict(comment)[:,1]
print(pm.shape)
print(pm)

#pm.plot().show()
plt.plot(range(len(text)),pm)
plt.xticks(range(len(text)), [x for x in text])
plt.show()
# = debias_attention_model.probs_model.predict(comment)
#print(b)
#reverse_word_map = dict(map(seq, debias_attention_model.tokenizer.word_index.items()))
# TODO you might have to get the word back from the array

[ 0.21407419  0.13594475  0.27313101  0.13851555  0.30889753  0.09868281
  0.21407419]
7


AttributeError: AttentionToxModel instance has no attribute 'probs_model'

In [55]:
layer = debias_attention_model.model.layers[2]
print(dir(layer))
print(layer.weights)
print(layer.get_config)
print(layer.outbound_nodes)
print(layer.output[0])
model = debias_attention_model


['__call__', '__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_inbound_node', '_built', '_get_node_attribute_at_index', '_initial_weights', '_losses', '_node_key', '_non_trainable_weights', '_per_input_losses', '_per_input_updates', '_trainable_weights', '_updates', 'activation', 'activity_regularizer', 'add_loss', 'add_update', 'add_weight', 'assert_input_compatibility', 'bias', 'bias_constraint', 'bias_initializer', 'bias_regularizer', 'build', 'built', 'call', 'compute_mask', 'compute_output_shape', 'count_params', 'data_format', 'dilation_rate', 'filters', 'from_config', 'get_config', 'get_input_at', 'get_input_mask_at', 'get_input_shape_at', 'get_losses_for', 'get_output_at', 'get_output_mask_at', 'get_output_shape_at', 'get_updates_for', 'get_weights', 'inbound_nodes', 'input

In [ ]:
import keras


### Random model

In [92]:
random_test = pd.read_csv(random['test'])
#model.score_auc(random_test['comment'], random_test['is_toxic'])
print(set(random_test['is_toxic']))

set([False, True])


In [133]:
MODEL_NAME = 'cnn_debias_random_tox_v3'
debias_random_model = ToxModel()
debias_random_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.16800, saving model to ../models/cnn_debias_random_tox_v3_model.h5
9s - loss: 0.2299 - acc: 0.9201 - val_loss: 0.1680 - val_acc: 0.9392
Epoch 2/20
Epoch 00001: val_loss improved from 0.16800 to 0.14852, saving model to ../models/cnn_debias_random_tox_v3_model.h5
8s - loss: 0.1594 - acc: 0.9418 - val_loss: 0.1485 - val_acc: 0.9460
Epoch 3/20
Epoch 00002: val_loss improved from 0.14852 to 0.14

AttributeError: ToxModel instance has no attribute 'probs_model'

In [ ]:
random_test = pd.read_csv(random['test'])
debias_random_model.score_auc(random_test['comment'], random_test['is_toxic'])

0.96087133494510768

### Plain wikipedia model

In [ ]:
MODEL_NAME = 'cnn_wiki_tox_v3'
wiki_model = ToxModel()
wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 95692 samples, validate on 32128 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.19048, saving model to ../models/cnn_wiki_tox_v3_model.h5
8s - loss: 0.2477 - acc: 0.9105 - val_loss: 0.1905 - val_acc: 0.9340
Epoch 2/20
Epoch 00001: val_loss improved from 0.19048 to 0.15330, saving model to ../models/cnn_wiki_tox_v3_model.h5
8s - loss: 0.1690 - acc: 0.9383 - val_loss: 0.1533 - val_acc: 0.9431
Epoch 3/20


In [ ]:
wiki_test = pd.read_csv(wiki['test'])
wiki_model.score_auc(wiki_test['comment'], wiki_test['is_toxic'])

### Debiased model

In [8]:
MODEL_NAME = 'cnn_debias_tox_v3'
debias_model = ToxModel()
debias_model.train(debias['train'], debias['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/6
99157/99157 [==============================] - 195s - loss: 0.2072 - acc: 0.9262 - val_loss: 0.1501 - val_acc: 0.9456
Epoch 2/6
99157/99157 [==============================] - 194s - loss: 0.1459 - acc: 0.9473 - val_loss: 0.1302 - val_acc: 0.9519
Epoch 3/6
99157/99157 [==============================] - 199s - loss: 0.1305 - acc: 0.9527 - val_loss: 0.1378 - val_acc: 0.9523
Epoch 4/6
99157/99157 [==============================] - 190s - loss: 0.1210 - acc: 0.9562 - val_loss: 0.1204 - val_acc: 0.9550
Epoch 5/6
99157/99157 [==============================] - 188s - loss: 0.1134 - acc: 0.9591 - val_loss: 0.1222 - val_acc: 0.9562
Epoch 6/6
99157/99157 [==============================] - 189s - loss: 0.1063 - acc: 0.9611 - val_loss: 0.1151 - val_acc: 0.9572
Model trained!
Saving model...
Model saved!


In [8]:
debias_test = pd.read_csv(debias['test'])
debias_model.prep_data_and_score(debias_test['comment'], debias_test['is_toxic'])

0.97214632823959757

In [ ]:
MODEL_NAME = 'attention_bias_tox_v3'
debias_model = AttentionToxModel()
debias_model.train(debias['train'], debias['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.22884, saving model to ../models/attention_bias_tox_v3_model.h5
101s - loss: 0.2936 - acc: 0.9067 - val_loss: 0.2288 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved from 0.22884 to 0.19191, saving model to ../models/attention_bias_tox_v3_model.h5
101s - loss: 0.2090 - acc: 0.9144 - val_loss: 0.1919 - val_acc: 0.9321
Epoch 3/20
Epoch 00002: val_loss improv